<a href="https://colab.research.google.com/github/princetyagitech/NLP/blob/master/KeyWord_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy
import nltk
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path='/content/drive/My Drive/nips-papers.zip'
path1='/content/drive/My Drive'
import os
#os.listdir(path)
import zipfile
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall(path1)
zip_ref.close()

In [ ]:
papers=pd.read_csv(path1+'/papers.csv')
paper_authors=pd.read_csv(path1+'/paper_authors.csv')
authors=pd.read_csv(path1+'/authors.csv')

In [ ]:
dataset=papers[papers['abstract']!='Abstract Missing'].reset_index()

In [ ]:
len(dataset)

3924

In [ ]:
dataset.head()

,index,id,year,title,event_type,pdf_name,abstract,paper_text
0,941,1861,2000,Algorithms for Non-negative Matrix Factorization,NaN,1861-algorithms-for-non-negative-matrix-factor...,Non-negative matrix factorization (NMF) has pr...,Algorithms for Non-negative Matrix\nFactorizat...
1,1067,1975,2001,Characterizing Neural Gain Control using Spike...,NaN,1975-characterizing-neural-gain-control-using-...,Spike-triggered averaging techniques are effec...,Characterizing neural gain control using\nspik...
2,2384,3163,2007,Competition Adds Complexity,NaN,3163-competition-adds-complexity.pdf,It is known that determinining whether a DEC-P...,Competition adds complexity\n\nJudy Goldsmith\...
3,2385,3164,2007,Efficient Principled Learning of Thin Junction...,NaN,3164-efficient-principled-learning-of-thin-jun...,We present the first truly polynomial algorith...,Efficient Principled Learning of Thin Junction...
4,2388,3167,2007,Regularized Boost for Semi-Supervised Learning,NaN,3167-regularized-boost-for-semi-supervised-lea...,Semi-supervised inductive learning concerns ho...,Regularized Boost for Semi-Supervised Learning...


# Text Pre-Processing
Main aim is to remove unwanted punctuations ,digits and convert words to stem words and normalizing the text

### Finding out average length of every abstract

In [ ]:
dataset['wordcount']=dataset['abstract'].apply(lambda x: len(str(x).split(' ')))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dataset[['abstract','wordcount']].head()

,abstract,wordcount
941,Non-negative matrix factorization (NMF) has pr...,107
1067,Spike-triggered averaging techniques are effec...,81
2384,It is known that determinining whether a DEC-P...,67
2385,We present the first truly polynomial algorith...,143
2388,Semi-supervised inductive learning concerns ho...,119


In [ ]:
dataset.wordcount.describe()

count    3924.000000
mean      148.390928
std        45.605755
min        19.000000
25%       116.000000
50%       143.000000
75%       177.000000
max       317.000000
Name: wordcount, dtype: float64

So average length of column is 148 and max is 317,min is 19.

### Most Common and Uncommon Words in corpus of text

In [ ]:
#Identify common words
common = pd.Series(' '.join(dataset['abstract']).split()).value_counts()[:20]
print(common)
uncommon=pd.Series(' '.join(dataset['abstract']).split()).value_counts()[-20:]
print(uncommon)

the         29793
of          20918
a           16339
and         13626
to          12869
in           8980
that         7838
is           7666
for          7169
We           6238
on           5579
we           5167
with         4512
this         3677
as           3643
are          3529
an           3366
by           3197
can          2953
learning     2825
dtype: int64
gFIC.                 1
m}$,                  1
fixed-step-size       1
grammars.             1
(KMC),                1
backlog               1
$f(x_1(u),            1
$|z                   1
Context-Sensitive     1
\emph{new}            1
multi-Gigabyte        1
meaningful"           1
$\Pi$                 1
surged                1
readers.              1
learning-theoretic    1
$i=1,2,\ldots,m$,     1
neglected             1
Autism                1
$\ps$                 1
dtype: int64


Looks like most uncommon words are actually noise in the dataset which we need to remove.

### Removing stopwords from the corpus


In [ ]:
stop_words = set(stopwords.words("english"))


In [ ]:
dataset['abstract'][941]

'Non-negative matrix factorization (NMF) has previously been shown to \r\nbe a useful decomposition for multivariate data. Two different multi- \r\nplicative algorithms for NMF are analyzed. They differ only slightly in \r\nthe multiplicative factor used in the update rules. One algorithm can be \r\nshown to minimize the conventional least squares error while the other \r\nminimizes the generalized Kullback-Leibler divergence. The monotonic \r\nconvergence of both algorithms can be proven using an auxiliary func- \r\ntion analogous to that used for proving convergence of the Expectation- \r\nMaximization algorithm. The algorithms can also be interpreted as diag- \r\nonally rescaled gradient descent, where the rescaling factor is optimally \r\nchosen to ensure convergence. '

In [ ]:
corpus=[]
for i in range(len(dataset)):
  #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', dataset['abstract'][i])
    
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    ##Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    corpus.append(text)
  

In [ ]:
corpus[0]

'non negative matrix factorization nmf useful decomposition multivariate data different multi plicative algorithm nmf analyzed differ slightly multiplicative factor used update rule algorithm minimize conventional least square error minimizes generalized kullback leibler divergence monotonic convergence algorithm proven auxiliary func tion analogous used proving convergence expectation maximization algorithm algorithm interpreted diag onally rescaled gradient descent rescaling factor optimally chosen ensure convergence'

# Creating a vector of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re
cv=CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=10000, ngram_range=(1,3))
X=cv.fit_transform(corpus)

In [ ]:
list(cv.vocabulary_.keys())[:10]

['non',
 'negative',
 'matrix',
 'factorization',
 'nmf',
 'useful',
 'decomposition',
 'multivariate',
 'data',
 'different']

# TF-IDF Matrix

In [ ]:
#Function for sorting tf_idf in descending order
from scipy.sparse import coo_matrix
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(X)
# get feature names
feature_names=cv.get_feature_names()
keywordslist=[]
for i in range(len(corpus)):
  keyl=[]
  #generate tf-idf for the given document
  tf_idf_vector=tfidf_transformer.transform(cv.transform([corpus[i]]))
  #sort the tf-idf vectors by descending order of scores
  sorted_items=sort_coo(tf_idf_vector.tocoo())
  #extract only the top n; n here is 10
  keywords=extract_topn_from_vector(feature_names,sorted_items,5)
  keyl=keywords
  keywordslist.append(keyl)

In [ ]:
keywordslist[:5]

[{'algorithm': 0.18,
  'algorithm minimize': 0.156,
  'convergence': 0.208,
  'factor': 0.162,
  'nmf': 0.288},
 {'gain': 0.2,
  'gain control': 0.313,
  'spike': 0.195,
  'spike triggered': 0.294,
  'triggered': 0.278},
 {'complete': 0.27,
  'cooperative': 0.347,
  'expected reward': 0.37,
  'reward': 0.24,
  'whether': 0.26},
 {'junction': 0.238,
  'junction tree': 0.249,
  'mutual information': 0.18,
  'polynomial': 0.209,
  'treewidth': 0.327},
 {'boosting algorithm': 0.281,
  'local smoothness': 0.348,
  'regularizer': 0.323,
  'semi': 0.259,
  'semi supervised': 0.283}]